In [43]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

In [53]:
df = pd.read_excel('House_test.xlsx')

In [54]:
# biến 'Dien_tich' từ object sang dạng float
lst=df['Dien_tich'][df['Dien_tich'].str.contains('\d+\.\d{3}\.')==True].str.split('\.',expand=True).astype(str)
lst['ghep']=(lst[0]+lst[1]+'.'+lst[2]).astype(float)
df['Dien_tich'][df['Dien_tich'].str.contains('\d+\.\d{3}\.')==True]=lst['ghep']
print(df['Dien_tich'][df['Dien_tich'].str.contains('\d+\.\d{3}\.')==True])
df[['Dien_tich','Dien_tich_su_dung']]=df[['Dien_tich','Dien_tich_su_dung']].astype(float)
# extract ra biến giá đơn vị triệu đồng
df['Gia_ty']=df['Gia'].str.extract('(.+) tỷ \d+')
df['Gia_trieu_đong']=df['Gia_ty'].str.replace(',','.').astype(float)*1000### ko có str nó se ko replace
df['Gia']=df['Gia'].str.replace(',','.')
df['Don_gia_trieu']=df['Gia'].str.extract('[Đã bán|triệu] (\d+\.?\d+) triệu/m²').astype(float)
ind_gia=df.Gia_trieu_đong[df.Gia_trieu_đong.isnull()==True].index
df.loc[ind_gia,'Gia_trieu_đong']=df.loc[ind_gia,'Don_gia_trieu']*df.loc[ind_gia,'Dien_tich']
# biến C2 là số lầu của ngôi nhà, extract từ biến 'Cau_truc'
df['C2']=df['Cau_truc'].str.extract('(\d+) lầu')#.astype(int)# lúc đầu thiếu \ nên lỗi ko đc
df['C2'].fillna(0,inplace=True)
df['C2']=df['C2'].astype(int)
# Xử lý ván đề Hem_rong=0 tức là có mặt đường: tăng hệ só cho biến mặt đường roi thay  thế các giá trị mặt đường =0 bằng biến hẻm rộng
df['Duong_mat_tien']=df['Duong_mat_tien']*1.8
ind=df[df['Duong_mat_tien']==0].index
df.loc[ind,'Duong_mat_tien']=df.loc[ind,'Hem_rong']
# những giá trị không phải là số của biến 'So_phong', thay bằng median
df['So_phong'][df['So_phong'] =='  --  ']=np.nan
df['So_phong'][df['So_phong'] =='--']=np.nan
df['So_phong']=df['So_phong'].astype(float)
df['So_phong'].fillna(round(df['So_phong'][df['So_phong'].notnull()].median(),0),inplace=True)
# tạo Dummy cho biến Quan
df_new = pd.get_dummies(data = df, columns=['Quan'], prefix='Dum')###
X = df_new[['So_phong', 'Dien_tich', 'Duong_mat_tien','C2','Dum_ Huyện Bình Chánh', 'Dum_ Huyện Hóc Môn', 'Dum_ Huyện Nhà Bè',
       'Dum_ Quận 1', 'Dum_ Quận 10', 'Dum_ Quận 11', 'Dum_ Quận 12',
       'Dum_ Quận 2', 'Dum_ Quận 3', 'Dum_ Quận 4', 'Dum_ Quận 5',
       'Dum_ Quận 6', 'Dum_ Quận 7', 'Dum_ Quận 8',
       'Dum_ Quận Bình Thạnh', 'Dum_ Quận Bình Tân', 'Dum_ Quận Gò Vấp',
       'Dum_ Quận Phú Nhuận', 'Dum_ Quận Tân Bình']]
y = df['Gia_trieu_đong']


Series([], Name: Dien_tich, dtype: object)


In [55]:
import pickle
def Load_object(filename): 
     with open(filename, 'rb') as file:  
          obj=pickle.load(file)
     return obj 
model=Load_object('pipe.pkl')

In [56]:
model.fit(X, y)
yhat= model.predict(X)
print('R-squared:', model.score(X,y))
a=np.sqrt(((yhat-y) ** 2).mean())
print('rmse',a)

R-squared: 0.863461893531465
rmse 3878.624358312651
